In [94]:
import os
from dataclasses import dataclass
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import countries, dap_bidding_zones, interconnections
from tqdm import tqdm
from dotenv import load_dotenv

In [13]:
load_dotenv()
engine = create_engine(os.getenv("SQLALCHEMY_DATABASE_URI"))

In [14]:
@dataclass
class Node:
    name: str
    code: str
    load_df: pd.DataFrame
    gen_df: pd.DataFrame
    dap_df: pd.DataFrame


nodes = {
    country_id: Node(
        name=country_id,
        code=domain,
        load_df=None,
        gen_df=None,
        dap_df=None,
    )
    for country_id, domain in countries.items()
}

In [15]:
for country_id, domain in tqdm(countries.items()):
    load_df = pd.read_sql_query(
        f"""SELECT "DateTime", "TotalLoadValue" 
        FROM load WHERE "AreaCode" = '{domain}' AND "AreaTypeCode" = 'CTY' AND "DateTime" >= '2015-01-01 00:00:00' AND "DateTime" <= '2020-01-01 00:00:00'
        ORDER BY "DateTime";""",
        engine,
        parse_dates=["DateTime"],
        index_col="DateTime",
    )
    assert load_df.index.is_unique
    assert load_df.index.is_monotonic_increasing
    load_df = load_df.resample("1h").mean()
    nodes[country_id].load_df = load_df

100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


In [16]:
for country_id, domain in tqdm(countries.items()):
    gen_df = pd.read_sql_query(
        f"""SELECT "DateTime", "ProductionType", "ActualGenerationOutput"
        FROM generation WHERE "AreaCode" = '{domain}' AND "AreaTypeCode" = 'CTY' AND "DateTime" >= '2015-01-01 00:00:00' AND "DateTime" <= '2020-01-01 00:00:00'
        ORDER BY "DateTime";""",
        engine,
        parse_dates=["DateTime"],
    )
    gen_df = gen_df.pivot_table(
        index="DateTime", columns="ProductionType", values="ActualGenerationOutput"
    )
    assert gen_df.index.is_unique
    assert gen_df.index.is_monotonic_increasing
    gen_df = gen_df.resample("1h").mean()
    nodes[country_id].gen_df = gen_df

100%|██████████| 10/10 [03:49<00:00, 22.98s/it]


In [71]:
for country_id, domain in tqdm(countries.items()):
    cty_dap_df = pd.DataFrame()
    for bzn in dap_bidding_zones[country_id]:
        if (
            bzn == "10Y1001A1001A59C"
        ):  # IE resolution changed midway through the dataset
            query = f"""SELECT "DateTime", "Price", "ResolutionCode"
            FROM dap WHERE "AreaCode" = '{bzn}' AND "AreaTypeCode" = 'BZN' AND "DateTime" >= '2015-01-01 00:00:00' AND "DateTime" <= '2020-01-01 00:00:00'
            ORDER BY "DateTime";"""
        else:
            query = f"""SELECT "DateTime", "Price" 
            FROM dap WHERE "AreaCode" = '{bzn}' AND "AreaTypeCode" = 'BZN' AND "ResolutionCode" = 'PT60M' AND "DateTime" >= '2015-01-01 00:00:00' AND "DateTime" <= '2020-01-01 00:00:00'
            ORDER BY "DateTime";"""
        bzn_dap_df = pd.read_sql_query(
            query,
            engine,
            parse_dates=["DateTime"],
            index_col="DateTime",
        )
        if bzn == "10Y1001A1001A59C":
            # pop index back into "DateTime" column
            bzn_dap_df.reset_index(inplace=True)
            bzn_dap_df = bzn_dap_df.sort_values(by=["DateTime", "ResolutionCode"])
            # for rows with the same DateTime, keep the row with ResolutionCode == "PT60M"
            bzn_dap_df = bzn_dap_df.drop_duplicates(subset="DateTime", keep="last")
            bzn_dap_df.drop(columns=["ResolutionCode"], inplace=True)
            bzn_dap_df.set_index("DateTime", inplace=True)

        assert bzn_dap_df.index.is_unique
        assert bzn_dap_df.index.is_monotonic_increasing
        bzn_dap_df = bzn_dap_df.resample("1h").mean()
        cty_dap_df = pd.concat([cty_dap_df, bzn_dap_df], axis=1)
    # Take the mean of the prices across bidding zones in the country
    cty_dap_df = cty_dap_df.mean(axis=1)
    cty_dap_df.columns = ["Price"]
    cty_dap_df = cty_dap_df.to_frame()
    nodes[country_id].dap_df = cty_dap_df

100%|██████████| 10/10 [00:11<00:00,  1.12s/it]


In [60]:
flow_df = pd.DataFrame()
for country_id, neighbour_ids in tqdm(interconnections.items()):
    from_domain = countries[country_id]
    to_domains = [
        (neighbour_id, countries[neighbour_id]) for neighbour_id in neighbour_ids
    ]

    for neighbour_id, to_domain in to_domains:
        # print(f"[{country_id}] Fetching flow data from {from_domain} to {to_domain}")
        ic_df = pd.read_sql_query(
            f"""SELECT "DateTime", "FlowValue" 
            FROM flow WHERE "OutAreaCode" = '{from_domain}' AND "InAreaCode" = '{to_domain}' AND "OutAreaTypeCode" = 'CTY' AND "InAreaTypeCode" = 'CTY' AND "DateTime" >= '2015-01-01 00:00:00' AND "DateTime" <= '2020-01-01 00:00:00'
            ORDER BY "DateTime";""",
            engine,
            parse_dates=["DateTime"],
            index_col="DateTime",
        )
        assert ic_df.index.is_unique
        assert ic_df.index.is_monotonic_increasing
        ic_df = ic_df.resample("1h").mean()
        ic_df = ic_df.rename(columns={"FlowValue": f"{country_id}->{neighbour_id}"})
        flow_df = pd.concat([flow_df, ic_df], axis=1)

100%|██████████| 10/10 [03:25<00:00, 20.52s/it]


In [90]:
for country_id, domain in tqdm(countries.items()):
    print(
        country_id,
        nodes[country_id].load_df.shape,
        nodes[country_id].gen_df.shape,
        nodes[country_id].dap_df.shape,
        flow_df.shape,
    )

100%|██████████| 10/10 [00:00<?, ?it/s]

BE (43824, 1) (82072, 18) (43729, 1) (43824, 32)
DK (43824, 1) (81328, 18) (43824, 1) (43824, 32)
FR (43824, 1) (81729, 18) (43729, 1) (43824, 32)
DE (43824, 1) (81330, 18) (43729, 1) (43824, 32)
IE (43824, 1) (81343, 18) (43818, 1) (43824, 32)
LU (43824, 1) (81330, 18) (43729, 1) (43824, 32)
NL (43824, 1) (81954, 18) (43729, 1) (43824, 32)
NO (43824, 1) (81761, 18) (43824, 1) (43824, 32)
CH (43824, 1) (81353, 18) (43824, 1) (43824, 32)
UK (43824, 1) (80906, 18) (43824, 1) (43824, 32)


In [87]:
for country_id, domain in tqdm(countries.items()):
    print(list(nodes[country_id].gen_df.columns))

100%|██████████| 10/10 [00:00<?, ?it/s]

['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore', 'Geothermal', 'Fossil Peat']
['Biomass', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore', 'Hydro Run-of-river and poundage', 'Hydro Pumped Storage', 'Hydro Water Reservoir', 'Nuclear', 'Other', 'Other renewable', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Geothermal', 'Fossil Peat']
['Biomass', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Nuclear', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore', 'Other', 'Other renewable', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Geothermal', 'Fossil Peat']
['Biomass', 'Fossil Brown coal/Lignite', 'F

In [88]:
# Find the common column names of gen_df across all nodes
common_gen_cols = set(nodes["BE"].gen_df.columns)
for node in nodes.values():
    common_gen_cols = common_gen_cols.intersection(set(node.gen_df.columns))
common_gen_cols = list(common_gen_cols)
common_gen_cols.sort()
print(common_gen_cols)

['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']


In [89]:
gen_cols = {}
for country_id, domain in countries.items():
    cols = list(nodes[country_id].gen_df.columns)
    for col in cols:
        if col not in gen_cols:
            gen_cols[col] = 1
        else:
            gen_cols[col] += 1
gen_cols = {
    k: v for k, v in sorted(gen_cols.items(), key=lambda item: item[1], reverse=True)
}
print(len(gen_cols))
for k, v in gen_cols.items():
    print(k, v)

18
Biomass 10
Fossil Brown coal/Lignite 10
Fossil Coal-derived gas 10
Fossil Gas 10
Fossil Hard coal 10
Fossil Oil 10
Hydro Pumped Storage 10
Hydro Run-of-river and poundage 10
Hydro Water Reservoir 10
Nuclear 10
Other 10
Other renewable 10
Solar 10
Waste 10
Wind Offshore 10
Wind Onshore 10
Geothermal 10
Fossil Peat 10


In [86]:
for country_id, domain in countries.items():
    cols = list(nodes[country_id].gen_df.columns)
    for gen_col in gen_cols.keys():
        if gen_col not in cols:
            nodes[country_id].gen_df[gen_col] = np.nan

In [91]:
# Save the data into database
for country_id, node in nodes.items():
    node.load_df.to_sql(f"{country_id}_load", engine, if_exists="replace")
    node.gen_df.to_sql(f"{country_id}_gen", engine, if_exists="replace")
    node.dap_df.to_sql(f"{country_id}_dap", engine, if_exists="replace")
flow_df.to_sql("flow_32", engine, if_exists="replace")

264